<div style="line-height:1.2;">

<h1 style="color:#FF7C00; margin-bottom: 0.3em;">Tensorflow basics 2 </h1>

<h4 style="margin-top: 0.3em; margin-bottom: 1em;"> Other Tensors and Neural Networks examples.</h4>

<div style="line-height:1.4; margin-bottom: 0.5em;">
    <h3 style="color: lightblue; display: inline; margin-right: 0.5em;">Keywords:</h3>
    jedi completion in Colab + tf.dtype + @tf.function decorator + tf.math + tf.constant + keras
</div>

</div>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
%config Completer.use_jedi = True
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
#%%script echo Skipping, just to check GPU
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
!nvidia-smi

Mon Nov  6 16:05:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    13W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
one_tens = tf.constant(np.arange(1, 10))
one_tens

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [6]:
dtype_of_elements_in_one_tens = one_tens.dtype
dtype_of_elements_in_one_tens

tf.int64

In [7]:
squa = tf.square(one_tens)
squa

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

### => sqrt, take the square root of each tensor's value

In [8]:
""" sqrt 1
N.B.
sq = tf.sqrt(one_tens) #Error!
--> InvalidArgumentError: allowed values: bfloat16, half, float, double, complex64, complex128
"""
## To use the same one_tens
casted = tf.cast(one_tens, dtype=tf.float32)
print(casted, end='\n\n')
print(tf.sqrt(casted))

tf.Tensor([1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(9,), dtype=float32)

tf.Tensor(
[1.        1.4142135 1.7320508 2.        2.2360678 2.4494896 2.6457512
 2.828427  3.       ], shape=(9,), dtype=float32)


In [9]:
""" sqrt 2. Create same tensor in different way """
another_tens = tf.constant(np.arange(1, 10, dtype=np.float32))
sq = tf.sqrt(another_tens)

print(another_tens, end='\n\n')
print(sq)

tf.Tensor([1. 2. 3. 4. 5. 6. 7. 8. 9.], shape=(9,), dtype=float32)

tf.Tensor(
[1.        1.4142135 1.7320508 2.        2.2360678 2.4494896 2.6457512
 2.828427  3.       ], shape=(9,), dtype=float32)


In [10]:
dtype_of_elements_in_sq = sq.dtype
dtype_of_elements_in_sq

tf.float32

### => log

In [11]:
""" Find the log
N.B.
Using 'tf.math.log(one_tens)' leads to error: InvalidArgumentError.
The tensor need to contains floats!
"""
# Convert to float32
one_tens_float = tf.cast(one_tens, dtype=tf.float32)
# Display only the content of the tensor
tf.print(one_tens_float)
one_tens_float

[1 2 3 ... 7 8 9]


<tf.Tensor: shape=(9,), dtype=float32, numpy=array([1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)>

In [12]:
log_tens = tf.math.log(one_tens_float)
log_tens

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### => slicing

In [13]:
""" Create a 2D tensor """
a_matrix = [[1,2,3,4,5],
        [6,7,8,9,10],
        [11,12,13,14,15],
        [16,17,18,19,20]]

tensor = tf.Variable(a_matrix, dtype=tf.int32)
print(tf.rank(tensor))
print(tensor.shape)
# Show the content only
tf.print(tensor)

tf.Tensor(2, shape=(), dtype=int32)
(4, 5)
[[1 2 3 4 5]
 [6 7 8 9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]]


In [14]:
# Select the first row
row1 = tensor[0]
tf.print(row1)
# Select the third column
column3 = tensor[:, 0]
tf.print(column3)
# Select the 3rd element from the 1st row
three = tensor[0, 2]
tf.print(three, end='\n\n')            #adding blank line works also here!
# Select second and fourth row
row_2_and_4 = tensor[1::2]
tf.print(row_2_and_4, end= '\n\n')
# Select first columns + second and third row
column_1_in_row_2_and_3 = tensor[1:3, 0]
tf.print(column_1_in_row_2_and_3)

[1 2 3 4 5]
[1 6 11 16]
3

[[6 7 8 9 10]
 [16 17 18 19 20]]

[6 11]


In [15]:
""" Differently from PyTorch, tf tensors are immutable!
Therefore, when created using a slice of another tensor, they do not share the same underlying data.
There is no concepts as views, so any modifications made to the original tensor do not affect the new tensor created via slicing, and vice versa.
For instance, the "assign" method, does not modify the tensor in-place; on the contrary it creates a new tensor with the updated values and
then assigns it to the variable => does not affect the slices that have already been extracted!
"""
tensor[0, 2].assign(999)
tf.print(tensor, end='\n\n')
tf.print(three)

[[1 2 999 4 5]
 [6 7 8 9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]]

3


<h2 style="color:#FF7C00  "> Manipulate tf.Variable </h2>

In [16]:
ten2 = tf.Variable(np.arange(0, 5))
ten2

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([0, 1, 2, 3, 4])>

In [17]:
# Show only the content
print(ten2.numpy())
ten2.numpy()

[0 1 2 3 4]


array([0, 1, 2, 3, 4])

In [18]:
# Assign the final value a new value of 50
ten2.assign([0, 1, 2, 3, 50])
ten2

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([ 0,  1,  2,  3, 50])>

In [19]:
# Add 10 to every element
ten2.assign_add([10, 10, 10, 10, 10])
ten2

<tf.Variable 'Variable:0' shape=(5,) dtype=int64, numpy=array([10, 11, 12, 13, 60])>

<h3 style="color:#FF7C00  "> Tf with Numpy </h3>

In [20]:
## Tensor from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [21]:
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [22]:
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [23]:
## Create a tensor from NumPy and from an array
numpy_J = tf.constant(np.array([3., 7., 10.]))  #float64
tensor_J = tf.constant([3., 7., 10.])           #float32
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

<h2 style="color:#FF7C00  "> decorator @tf.function </h2>

In [24]:
def func(x, y):
  return x ** 2 + y

x = tf.constant(np.arange(0, 10))
y = tf.constant(np.arange(10, 20))
func(x, y)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 10,  12,  16,  22,  30,  40,  52,  66,  82, 100])>

In [25]:
## Same function and decorate it with tf.function
@tf.function
def tf_function(x, y):
  return x ** 2 + y

tf_function(x, y)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 10,  12,  16,  22,  30,  40,  52,  66,  82, 100])>

In [26]:
""" Function that will be converted to a graph function. """
@tf.function
def add(a, b):
    return tf.add(a, b)

a = tf.constant([1, 2, 3])
b = tf.constant([4, 5, 6])

## Graph function
c = add(a, b)
c

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 7, 9], dtype=int32)>

In [27]:
real = np.random.rand(3, 3)
imag = np.random.rand(3, 3)
complex_tens = tf.complex(real, imag)

# compute the absolute value of the complex tensor
abs_tens = tf.abs(complex_tens)
# compute the conjugate of the complex tensor
conj_tens = tf.math.conj(complex_tens)

print(real)
print(imag, end='\n\n')
print(abs_tens)
print(conj_tens)

[[0.6255419  0.06785149 0.33478425]
 [0.5744087  0.64301323 0.72131303]
 [0.00526587 0.84347421 0.17834122]]
[[0.94217079 0.5302481  0.05626843]
 [0.86222438 0.37396378 0.04080237]
 [0.30072075 0.61736787 0.70428927]]

tf.Tensor(
[[1.13092372 0.53457167 0.33947994]
 [1.03603872 0.74385141 0.72246614]
 [0.30076685 1.04527117 0.72651838]], shape=(3, 3), dtype=float64)
tf.Tensor(
[[0.6255419 -0.94217079j 0.06785149-0.5302481j  0.33478425-0.05626843j]
 [0.5744087 -0.86222438j 0.64301323-0.37396378j 0.72131303-0.04080237j]
 [0.00526587-0.30072075j 0.84347421-0.61736787j 0.17834122-0.70428927j]], shape=(3, 3), dtype=complex128)


<h2 style="color:#FF7C00  "> Load datasets </h2>

<h3 style="color:#FF7C00  "> Note: </h3>
<div style="margin-top: -10px;">
The Reuters-21578 dataset is a collection of documents with news articles. The original corpus has 10,369 documents and a vocabulary of 29,930 words.
</div>

In [28]:
# Load the dataset, specifying the number of most frequent words to keep
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.reuters.load_data(num_words=10000)

# Get the word index mapping
word_index = tf.keras.datasets.reuters.get_word_index()

for i, (key, value) in enumerate(word_index.items()):
    if i == 10:
        break
    print(f"{key}: {value}")

mdbl: 10996
fawc: 16260
degussa: 12089
woods: 8803
hanging: 13796
localized: 20672
sation: 20673
chanthaburi: 20675
refunding: 10997
hermann: 8804


In [29]:
type(x_train), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [30]:
x_train[:3]

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]),
       list([1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12]),
       list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110,

In [31]:
# Reverse the word index to get back to text
reverse_word_index = {value: key for (key, value) in word_index.items()}
for i, (key, value) in enumerate(reverse_word_index.items()):
    if i == 10:
        break
    print(f"{key}: {value}")

10996: mdbl
16260: fawc
12089: degussa
8803: woods
13796: hanging
20672: localized
20673: sation
20675: chanthaburi
10997: refunding
8804: hermann


In [32]:
""" Decode the first newswire back into text.
N.B.1
The offset for indices (by 3) is necessary since the indices 0, 1, and 2 are reserved for "padding", "start of sequence", and "unknown".
N.B.2
Keras does not include a way to map from index to topic name, to retrieve
"""
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in x_train[0]])
decoded_newswire[:44]

'? ? ? said as a result of its december acqui'

<h2 style="color:#FF7C00  "> Create keras models </h2>

In [33]:
# Set the number of words to consider as features
max_features = 10000
# Define the maximum number of words to consider (the newswire cut-off length for the sequences)
maxlen = 200

# Load data in the feature and target sets
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.reuters.load_data(num_words=max_features)

<h3 style="color:#FF7C00  "> Note: </h3>
<div style="margin-top: -10px;">
The categorical_crossentropy loss expects labels to be provided in a one-hot encoded format, <br>
since it compares the predicted probability distribution (output of the softmax layer) against the true distribution, for each class.
</div>



In [34]:
""" Prepare data for training.
N.B.1
It is better to encode data
N.B.2
NumPy arrays must be converted to Tensors to avoid a ValueError.
N.B.3
All input sequences should be of the same length => layers expect uniform input shapes. One std approach to solve this is use padding (adding zeros).
N.B.4
It is necessary to redo again the split !
"""
## Pad the sequences with zeros, to have the input sequences in a uniform length
x_train_full = pad_sequences(x_train_full, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Convert the labels to one-hot encoding
num_classes = np.max(y_train_full) + 1  # 46 topics
y_train_full = to_categorical(y_train_full, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [35]:
# Split the full training data into a training set and a validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)


In [36]:
""" Model definition => Initialize a Sequential model in Keras (a linear stack of layers).
# Layers tasks:
- 1 Encode the input sequence (positive integers) into a sequence of dense vectors of fixed size (output_dim=128)
- 2 Learn from sequence context and dependencies
- 3 Output the probability distribution over the 46 classes through a fully connected neural network layer
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(46, activation='softmax')  # 46 classes means 46 units
])

In [37]:
""" Model compilation.
N.B.
The "sparse-categorical Cross-Entropy" can be used for classification tasks with multiple classes where the labels are provided as integers.
"""
# Compile the model (prepare for training)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 46)                2990      
                                                                 
Total params: 1332398 (5.08 MB)
Trainable params: 1332398 (5.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
##### Train the model
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_data=(x_val, y_val)
)

Epoch 1/10
57/57 [==============================] - 18s 181ms/step - loss: 2.8228 - accuracy: 0.3450 - val_loss: 2.3706 - val_accuracy: 0.3534
Epoch 2/10
57/57 [==============================] - 7s 118ms/step - loss: 2.1459 - accuracy: 0.4326 - val_loss: 1.9164 - val_accuracy: 0.4919
Epoch 3/10
57/57 [==============================] - 5s 94ms/step - loss: 1.7813 - accuracy: 0.5222 - val_loss: 1.7583 - val_accuracy: 0.5470
Epoch 4/10
57/57 [==============================] - 5s 87ms/step - loss: 1.6349 - accuracy: 0.5637 - val_loss: 1.7294 - val_accuracy: 0.5426
Epoch 5/10
57/57 [==============================] - 4s 68ms/step - loss: 1.5580 - accuracy: 0.5756 - val_loss: 1.6457 - val_accuracy: 0.5799
Epoch 6/10
57/57 [==============================] - 4s 67ms/step - loss: 1.6043 - accuracy: 0.5669 - val_loss: 1.6274 - val_accuracy: 0.5843
Epoch 7/10
57/57 [==============================] - 4s 79ms/step - loss: 1.3635 - accuracy: 0.6306 - val_loss: 1.5443 - val_accuracy: 0.6060
Epoch 8/10